In [3]:
import os
os.chdir('../scripts')

In [15]:
import data_preparation as dp
import satisfactionAnalysis as sa
import experienceAnalysis as ea
import user_engagment as ue
from dotenv import load_dotenv
from importlib import reload


In [9]:
load_dotenv()

db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
db_name = os.getenv('DB_NAME')

db_url = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'
query = 'SELECT * FROM xdr_data'

In [11]:
reload(dp)
data = dp.load_data(db_url, query)
cleaned_data = dp.clean_data(data)

In [16]:
reload(ea)
reload(ue)

data_engagement = ue.aggregate_engagement_metrics(cleaned_data)
data_experience = ea.aggregate_user_experience(cleaned_data)

In [17]:
reload(sa)

engagement_columns = ['session_frequency', 'session_duration', 'total_traffic']
experience_columns = ['Average TCP Retransmission', 'Average RTT', 'Average Throughput']

data_engagement = sa.assign_engagement_score(data_engagement, engagement_columns)
data_experience = sa.assign_experience_score(data_experience, experience_columns)

data_engagement.head()
data_experience.head()

,MSISDN/Number,Average TCP Retransmission,Average RTT,Handset Type,Average Throughput,experience_cluster,experience_score
0,3.360300e+10,1.129040e+06,30.152653,Samsung Galaxy A5 Sm-A520F,4694.128998,0,3.567377
1,3.360301e+10,6.806669e+05,24.440917,Samsung Galaxy Note 8 (Sm-N950F),2217.750000,0,4.214480
2,3.360301e+10,1.267104e+06,33.381834,Samsung Galaxy S7 (Sm-G930X),7.500000,0,3.691911
3,3.360301e+10,1.267104e+06,11.000000,Apple iPhone X (A1901),57.500000,0,4.333990
4,3.360301e+10,1.267104e+06,13.500000,Apple iPhone 7 (A1778),33.500000,0,4.247378


In [18]:
data_satisfaction = sa.calculate_satisfaction_score(data_engagement, data_experience)
data_satisfaction.head()

,MSISDN/Number,engagement_score,experience_score,satisfaction_score
0,3.360300e+10,17.719698,3.567377,10.643537
1,3.360301e+10,0.008233,4.214480,2.111357
2,3.360301e+10,0.006761,3.691911,1.849336
3,3.360301e+10,0.016024,4.333990,2.175007
4,3.360301e+10,0.008092,4.247378,2.127735


In [20]:
reload(sa)

model = sa.build_regression_model(data_satisfaction)

Mean Squared Error: 6.902422996361739e-32


In [25]:
reload(sa)

data_clustered = sa.run_kmeans_clustering(data_satisfaction, n_clusters=2)
data_clustered.head()

,MSISDN/Number,engagement_score,experience_score,satisfaction_score,cluster
0,3.360300e+10,17.719698,3.567377,10.643537,0
1,3.360301e+10,0.008233,4.214480,2.111357,0
2,3.360301e+10,0.006761,3.691911,1.849336,0
3,3.360301e+10,0.016024,4.333990,2.175007,0
4,3.360301e+10,0.008092,4.247378,2.127735,0


In [28]:
reload(sa)

cluster_aggregation = sa.aggregate_scores_per_cluster(data_clustered)
print("Average Scores per Cluster: \n", cluster_aggregation)

Average Scores per Cluster: 
    cluster  engagement_score  experience_score  satisfaction_score
0        0          0.017250          3.736871            1.877060
1        1        517.969405          3.222594          260.595999


In [29]:
table_name = 'user_satisfaction'

sa.export_to_mysql(data_clustered, db_url, table_name)

print("Data exported to MySQL database")

Data exported to MySQL database
